# Doel van dit notebook
In dit experiment onderzoek ik of een Conv1D-laag vóór een GRU helpt bij gesture-classificatie.
Hypothese: lokale temporele patronen (acceleratie-pieken) kunnen door Conv1D beter worden geëxtraheerd voordat de RNN de sequentie samenvat.
Ik vergelijk dit model met de eerdere GRU + mean-pooling baseline, met identieke data- en trainingsinstellingen.

In [ ]:
import sys
from pathlib import Path

# Vind repo-root door omhoog te lopen totdat we pyproject.toml of .git vinden
p = Path().resolve()
while not (p / "pyproject.toml").exists() and not (p / ".git").exists():
    if p.parent == p:
        raise RuntimeError("Repo root not found (no pyproject.toml/.git).")
    p = p.parent

PROJECT_ROOT = p
sys.path.insert(0, str(PROJECT_ROOT))

print("Project root added to sys.path:", PROJECT_ROOT)

In [ ]:
from pathlib import Path

import torch

from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor

from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

In [ ]:
import torch

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Override naar CPU voor kleine RNN-modellen indien gewenst
device = torch.device("cpu")

device

In [ ]:
# Data + preprocessor (identiek aan notebook 02)
preprocessor = PaddedPreprocessor()

gestures_factory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gestures_factory.create_datastreamer(batchsize=32, preprocessor=preprocessor)

train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()

len(train), len(valid)

In [ ]:
accuracy = Accuracy()

settings = TrainerSettings(
    epochs=30,  # bewust korter dan "100": dit notebook is een gerichte vergelijking
    metrics=[accuracy],
    logdir=Path("gestures_ex3"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs={
        "save": False,
        "verbose": True,
        "patience": 5,
        "delta": 0.0,
    },
)
settings

In [ ]:
from src_rnn.models import RNNConfig
from src_rnn.run_experiment import run_experiment

In [ ]:
# # Run: Conv1D vóór GRU (experiment)
config = RNNConfig(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    output_size=20,
    dropout=0.0,
)

_ = run_experiment(
    model_name="conv1d_gru_mean",                 # baseline: last-step representatie
    config=config,
    settings=settings,
    trainstreamer=train.stream(),
    validstreamer=valid.stream(),
    device=device,
    experiment_name="gestures-ex3",
    run_name="03_conv1d_gru_mean_h64_l1",
)